In [1]:
%pip -q install google-genai
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [6]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"


# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response


# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


##########################################
# --- Agente 1: Buscador de Destinos --- #
##########################################
def agente_buscador(destino, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente de viagens. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para buscar informações relevantes sobre o destino abaixo.
        Informações tais como pontos turísticos, restaurantes, atrações, acomodações
        o que fazer no destino informado, melhor época do ano para visitar os destinos, etc.
        Foque em informações relevantes, com base na quantidade e entusiasmo das avaliações sobre ele.
        Se um ponto tiver poucas notícias ou avaliações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Essas informações relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        Inclua os links para os websites dos pontos turísticos, restaurantes, atrações, acomodações, etc. caso estejam disponíveis.
        """,
        description="Agente que busca informações no Google sobre determinado destino",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Destino: {destino}\nData de hoje: {data_de_hoje}"

    destino_buscado = call_agent(buscador, entrada_do_agente_buscador)
    return destino_buscado

################################################
# --- Agente 2: Planejador da viagem --- #
################################################
def agente_planejador(cidade_destino, estado_destino, cidade_origem, estado_origem):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um planejador de viagens, especialista em encontrar passagens aéreas e terrestres mais baratas,
        acomodações, restaurantes, passeios, etc. para um destino específico.
        Com base no destino relevantes buscado, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano de viagem para cada um deles,
        com valores de passagens aéres e terrestres, caso tenham. Você também pode usar o Google (google_search) para encontrar mais
        informações sobre o destino e aprofundar. Esse plano de viagen deverá ser completo e detalhado e deverá levar em consideração a cidade de origem
        e o estado de origem.
        Ao final, você irá escolher o plano de viagem mais relevante entre eles com base nas suas pesquisas
        e retornar esse plano de viagem completo, com seus pontos mais relevantes.
        """,
        description="Agente que planeja viagens",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Cidade destino:{cidade_destino}\nEstado destino: {estado_destino}\nCidade de Origem: {cidade_origem}\nEstado Origem: {estado_origem}"
    # Executa o agente
    plano_de_viagem = call_agent(planejador, entrada_do_agente_planejador)
    return plano_de_viagem

######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar e-mails marketing.
            Você escreve poste-mails para uma empresa de viagens especializada em destinos nacionais.
            Utilize o destino fornecido no plano de destino e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de um e-mail cativante sobre o tema indicado.
            O e-mail deve ser engajador, informativo, com linguagem simples e incluir uma chamada para ação no final.
            """,
        description="Agente redator de e-mails marketing engajadores"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho_gerado = call_agent(redator, entrada_do_agente_redator)
    return rascunho_gerado

##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Você tem um público diversificado, desde 18 anos até 70 anos. Use um tom de escrita adequado, envolvente, cativante,
            nem formal demais e nem coloquial demais.
            Revise o rascunho de post de Instagram abaixo sobre o destino indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado


data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
cidade_destino = input("❓ Por favor, informe a cidade de destino: ")
estado_destino = input("Informe o estado de destino: ")
cidade_origem = input("Informe a cidade de origem: ")
estado_origem = input("Informe o estado de origem: ")
destino = f"{cidade_destino}, {estado_destino} - {cidade_origem}, {estado_origem}"


########## Lógica do sistma de agentes ##########
if not destino:
    print("Você esqueceu de digitar o destino!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {cidade_destino} - {estado_destino}")

    destino_buscado = agente_buscador(destino, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(destino_buscado))
    print("--------------------------------------------------------------")

    plano_de_viagem = agente_planejador(cidade_destino, estado_destino, cidade_origem, estado_origem)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_viagem))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(destino, plano_de_viagem)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(destino, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, informe a cidade de destino: são joão del rey
Informe o estado de destino: mg
Informe a cidade de origem: rio de janeiro
Informe o estado de origem: rj
Maravilha! Vamos então criar o post sobre novidades em são joão del rey, mg - rio de janeiro, rj

--- 📝 Resultado do Agente 1 (Buscador) ---



> Ok, vou buscar informações relevantes sobre os destinos São João del Rei (MG) e Rio de Janeiro (RJ), incluindo pontos turísticos, restaurantes, atrações, acomodações e a melhor época para visitar, com base em avaliações recentes e informações atuais.
> 
> 
> ## São João del Rei, MG
> 
> **Pontos Turísticos:**
> 
> *   **Igreja São Francisco de Assis:** Construída em 1749, com estilo barroco e jardins projetados por Aleijadinho. No interior, altares esculpidos e arte sacra. A Igreja é um dos marcos mais emblemáticos da cidade.
> *   **Museu Ferroviário:** Localizado em uma antiga estação de trem, o museu preserva a história ferroviária da região. Possui uma coleção de locomotivas, vagões e documentos.
> *   **Museu de Arte Sacra:** Exibe uma coleção de objetos religiosos doados ao longo dos séculos.
> *   **Lagoa Azul:** Para amantes da natureza, a Lagoa Azul oferece águas cristalinas.
> 
> **Restaurantes:**
> 
> Não foi possível encontrar informações específicas e recentes sobre os melhores restaurantes em São João del Rei.
> 
> **Melhor Época para Visitar:**
> 
> *   A melhor época para visitar São João del Rei é de Abril a Setembro, quando o clima é mais ameno e seco.
> *   O outono (Março a Maio) oferece temperaturas amenas e clima mais seco, ideal para explorar museus e pontos históricos.
> *   Evite o verão (Novembro a Fevereiro) devido às temperaturas elevadas e chuvas frequentes.
> 
> ## Rio de Janeiro, RJ
> 
> **Pontos Turísticos:**
> 
> *   **Cristo Redentor:** Um dos principais símbolos da cidade, com vistas panorâmicas.
> *   **Escadaria Selarón:** Uma obra de arte vibrante que retrata a diversidade cultural do Rio.
> *   **Jardim Botânico e Floresta da Tijuca:** Ótimos para explorar a natureza da cidade.
> 
> **Restaurantes:**
> 
> Não foi possível encontrar informações específicas e recentes sobre os melhores restaurantes no Rio de Janeiro.
> 
> **Melhor Época para Visitar:**
> 
> *   A cidade é agradável o ano todo, mas a melhor época para evitar multidões e preços altos é entre Março e Junho, e entre Agosto e Novembro.
> *   **Outono (Março a Junho):** Temperaturas amenas e menos chuva.
> *   **Primavera (Setembro a Novembro):** Temperaturas em elevação e menos turistas.
> *   **Inverno (Junho a Setembro):** Dias mais secos e preços mais baixos.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Ok, entendi! Meu objetivo é criar um plano de viagem detalhado e completo do Rio de Janeiro para São João del Rei, incluindo opções de transporte (aéreo e terrestre), acomodação, restaurantes e passeios. Usarei a ferramenta de busca do Google para encontrar os melhores preços e informações relevantes.
> 
> 
> Com base nas informações que encontrei, aqui está um plano de viagem detalhado do Rio de Janeiro para São João del Rei:
> 
> **1. Transporte:**
> 
> *   **Ônibus:** Esta é a opção mais comum e acessível.
> 
>     *   **Preço:** As passagens de ônibus do Rio de Janeiro para São João del Rei variam entre R$ 147 e R$ 168. Os preços podem variar dependendo do dia, horário e tipo de ônibus.
>     *   **Empresas:** Viação Paraibuna.
>     *   **Horários:** Há diversas opções de horários, incluindo 07:00, 14:00 e 21:00.
>     *   **Duração:** A viagem de ônibus dura em média 5 horas e 30 minutos.
>     *   **Onde Comprar:** Você pode encontrar passagens nos sites BuscaOnibus, Buson, BlaBlaCar e CheckMyBus.
> *   **Passagens Aéreas:** Não há voos diretos do Rio de Janeiro para São João del Rei. Voos com escalas podem ser encontrados, mas podem não ser a opção mais eficiente em termos de tempo e custo.
> 
> **2. Acomodação:**
> 
> São João del Rei oferece diversas opções de hotéis e pousadas para todos os bolsos. Algumas opções bem avaliadas incluem:
> 
> *   **Hotéis:** Hotel Lenheiros, Terra D'Ouro Apart Hotel, Vicenza Apart Hotel, Garden Hill Hotel e Golfe.
> *   **Pousadas:** Pousada Paço do Lavradio, Pousada Villa Magnolia, Pousada Beco Do Bispo, Pousada Morada Do Sol.
> *   **Outras opções:** Hospedaria Casa Real, Solara Hotel, Hotel Cristo Rei 611.
> 
> **3. Restaurantes:**
> 
> São João del Rei oferece uma variedade de restaurantes com culinária mineira e outras opções. Alguns restaurantes recomendados incluem:
> 
> *   **Colher de Pau:** Comida mineira self-service.
> *   **Villeiros:** Comida a peso em um casarão do século XIX.
> *   **Monte Alverne:** Bem no centro da cidade, em frente à Igreja de São Francisco de Assis.
> *   **Outras opções:** Taberna D'Omar, Jardim Secreto da Oz, Carlotta Pizzeria.
> 
> **4. Pontos Turísticos:**
> 
> São João del Rei tem muitos pontos turísticos históricos e culturais. Alguns dos principais incluem:
> 
> *   Igreja de São Francisco de Assis.
> *   Catedral Basílica Nossa Senhora do Pilar.
> *   Igreja de Nossa Senhora do Carmo.
> *   Museu Regional de São João del Rei.
> *   Memorial Tancredo Neves.
> *   Passeio de Maria Fumaça e Museu Ferroviário.
> 
> **Plano de Viagem Sugerido:**
> 
> 1.  **Transporte:** Recomendo a viagem de ônibus, que é mais econômica e direta. A Viação Paraibuna oferece várias opções de horários.
> 2.  **Acomodação:** Para uma experiência completa, sugiro a Pousada Paço do Lavradio. Se busca algo mais econômico, o Hotel Lenheiros pode ser uma boa opção.
> 3.  **Alimentação:** Experimente a culinária mineira no Colher de Pau e o ambiente agradável do Villeiros.
> 4.  **Roteiro:**
> 
>     *   **Dia 1:** Visite a Igreja de São Francisco de Assis, o Museu Regional e o Memorial Tancredo Neves.
>     *   **Dia 2:** Faça o passeio de Maria Fumaça até Tiradentes e explore o Museu Ferroviário.
>     *   **Dia 3:** Conheça a Catedral Basílica Nossa Senhora do Pilar e a Igreja de Nossa Senhora do Carmo. Explore a Rua das Casas Tortas.
> 
> **Observações:**
> 
> *   Os preços e horários podem variar, então é sempre bom verificar com antecedência.
> *   São João del Rei é uma cidade histórica, então reserve um tempo para caminhar e apreciar a arquitetura.
> 
> Espero que este plano de viagem detalhado ajude você a aproveitar ao máximo sua visita a São João del Rei!


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> **Rascunho de Post para Instagram:**
> 
> ✨ RIO-MINAS: Uma Viagem Histórica Inesquecível! ✨
> 
> Direto do Rio de Janeiro para a charmosa São João del Rei! 🚌💨 Embarque em uma aventura de 5h30 de ônibus pela Viação Paraibuna (a partir de R$147!) e prepare-se para mergulhar na história e cultura mineira.
> 
> 🛌 **Onde Ficar?**
> *   Pousada Paço do Lavradio: puro charme!
> *   Hotel Lenheiros: conforto e economia.
> 
> 🍴 **Onde Comer?**
> *   Colher de Pau: comida mineira de dar água na boca!
> *   Villeiros: um casarão com sabores incríveis.
> 
> 🏛️ **O que Fazer?**
> *   Igreja de São Francisco de Assis: uma obra de arte!
> *   Maria Fumaça: uma viagem no tempo até Tiradentes.
> *   Catedral Basílica Nossa Senhora do Pilar: arquitetura deslumbrante.
> 
> Planeje sua rota e viva momentos inesquecíveis em São João del Rei! 🤩
> 
> #SãoJoãoDelRei #MinasGerais #RoteirosDeViagem #CulturaMineira
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está bom, mas podemos deixá-lo ainda mais atrativo com alguns ajustes:
> 
> **Problemas identificados:**
> 
> *   **Falta de impacto inicial:** O título atual é bom, mas pode ser mais chamativo para atrair a atenção imediatamente.
> *   **Tom um pouco genérico:** Algumas descrições podem ser mais específicas e envolventes para despertar o desejo de conhecer os lugares.
> *   **Chamada para ação:** Poderíamos adicionar um elemento que incentive mais o engajamento dos seguidores.
> 
> **Sugestões de melhorias:**
> 
> *   **Título:** Que tal algo como "Deixe o Rio e Apaixone-se por São João del Rei: Um Tesouro Mineiro!" ou "Rio-Minas Express: Descubra a Joia Histórica de São João del Rei"?
> *   **Descrição da viagem:** Adicione um toque pessoal! Por exemplo: "Deixe as praias cariocas por uns dias e venha se encantar com as ladeiras e a arquitetura colonial de São João del Rei. Uma viagem de 5h30 pela Viação Paraibuna (a partir de R$147) te leva a um mundo de história e sabores!"
> *   **Hospedagem:** "Pousada Paço do Lavradio: um refúgio de charme em um casarão histórico." "Hotel Lenheiros: conforto sem pesar no bolso, perfeito para explorar a cidade."
> *   **Gastronomia:** "Colher de Pau: prepare-se para uma explosão de sabores da autêntica cozinha mineira!" "Villeiros: um jantar especial em um casarão que te transporta para outra época."
> *   **Atividades:** "Igreja de São Francisco de Assis: admire a grandiosidade barroca e os detalhes que contam séculos de história." "Maria Fumaça: embarque em uma viagem nostálgica até Tiradentes, curtindo paisagens deslumbrantes." "Catedral Basílica Nossa Senhora do Pilar: perca-se na beleza da arquitetura e na riqueza dos detalhes em ouro."
> *   **Chamada para ação:** "Já visitou São João del Rei? Compartilhe sua dica favorita nos comentários! 👇" ou "Marque nos comentários quem precisa conhecer esse destino incrível! 👇"
> 
> Com essas pequenas mudanças, o post ficará ainda mais atraente e envolvente para o público do Instagram!


--------------------------------------------------------------
